# News recommendation serving
We will now implement a simple backend for our news recommendation example from earlier. It will integrate a flask webserver with stateful Ray actors to do serving.

### Implementing the backend with a Ray actor

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import atoma
from flask import Flask, jsonify, request
from flask_cors import CORS
import os
import requests
import ray

In [ ]:
ray.init(num_cpus=2)

We define an actor `NewsServer`, which is responsible for parsing an RSS feed and extracting the news items so they can be sent to the frontend. It also has a method `like_item`, which is called whenever the user "likes" and article. Note that this is a toy example, but in a more realistic applications, we could have a number of these actors, for example one for each user, to distribute the load.

In [ ]:
@ray.remote
class NewsServer(object):

    def __init__(self):
        pass

    def retrieve_feed(self, url):
        response = requests.get(url)
        feed = atoma.parse_rss_bytes(response.content)
        items = []
        for item in feed.items:
            color = "#FFFFFF" # white
            items.append({"title": item.title,
                          "link": item.link,
                          "description": item.description,
                          "description_text": item.description,
                          "pubDate": str(item.pub_date),
                          "color": color})

        return {"channel": {"title": feed.title,
                            "link": feed.link,
                            "url": feed.link},
                "items": items}

    def like_item(self, url, is_faved):
        if is_faved:
            print("url {} has been favorited".format(url))
        else:
            print("url {} has been defavorited".format(url))

### Deploying the backend

The following cell will set up a flask webserver that listens to commands from the frontend and dispatches them to the `NewsServer` actor.

In [ ]:
app = Flask("newsreader")
CORS(app)

@app.route("/api", methods=["POST"])
def dispatcher():
    req = request.get_json()
    method_name = req["method_name"]
    method_args = req["method_args"]
    if hasattr(dispatcher.server, method_name):
        method = getattr(dispatcher.server, method_name)
        # Doing a blocking ray.get right after submitting the task
        # might be bad for performance if the task is expensive.
        result = ray.get(method.remote(*method_args))
        return jsonify(result)
    else:
        return jsonify(
            {"error": "method_name '" + method_name + "' not found"})

dispatcher.server = NewsServer.remote()
app.run(host="0.0.0.0")

To try out the backend, evaluate the following cell and follow the resulting link.

In [ ]:
from IPython.core.display import HTML
if "JUPYTERHUB_SERVICE_PREFIX" in os.environ:
    url = "https://hub.mybinder.org" + os.environ["JUPYTERHUB_SERVICE_PREFIX"] + "proxy/9000/"
    html = "<a href=\"{}\">{}</a>".format(url)
else:
    html = "<b>(This feature is only available in Binder)</b>"
HTML(html)

You can then click on the "Add Channel" button and enter the URL of a newsfeed, for example `https://news.ycombinator.com/rss`. Click on one of the star icons and observe how the information is propagated to the Ray actor.

### Serving the model
We can now integrate the model we have trained in the `news_recommendation_training` example. First you need to click **Kernel/Restart & Clear Output** to prepare restarting the flask server. Then we import the needed modules and start Ray:

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import atoma
from flask import Flask, jsonify, request
from flask_cors import CORS
import os
import requests
import ray

In [ ]:
ray.init(num_cpus=2)

If you already completed this example, put the name of the file that generated the best model below and evaluate the following cell:

In [ ]:
best_result_path = ""
import pickle
with open(best_result_path, "rb") as f:
    pipeline = pickle.load(f)

**EXERCISE**: Copy the `NewsServer` code from above into the cell below and change it so that `color` will be

```python
color = "#FFD700" # gold-ish yellow
```

if `pipeline.predict([item.title])` is `True` and

```python
color = "#FFFFFF" # white
```

otherwise.

In [ ]:
# The new code for the NewsServer goes here

Now we can restart the flask server, open the news reader and look at the predictions.